In [24]:
import base64
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List, TypedDict , Dict
from dotenv import load_dotenv
load_dotenv()

True

## Pydantic Structures

In [25]:
# Ocr
class Ocr(BaseModel):
    question: str = Field(description="The question asked in the exam paper.")
    answer: str = Field(description="The student's answer text.")

# keypoints structure
class Introduction(BaseModel):
    marks: float
    criteria: str

class Dimension(BaseModel):
    title: str
    marks: float
    expected_elements: List[str]

class Body(BaseModel):
    marks: float
    dimensions: List[Dimension]

class Conclusion(BaseModel):
    marks: int 
    criteria: str

class GradingRubric(BaseModel):
    introduction: Introduction
    body: Body
    conclusion: Conclusion

# Root Model
class EvaluationStructure(BaseModel):
    question_text: str
    max_marks: float
    grading_rubric: GradingRubric


# answer evaluation structure
class MarksBreakdown(BaseModel):
    introduction: float = Field(alias="Introduction")
    body: float = Field(alias="Body")
    conclusion: float = Field(alias="Conclusion")
    total: float = Field(alias="Total")

class FeedbackDetail(BaseModel):
    introduction_comments: List[str] = Field(alias="Introduction")
    body_comments: List[str] = Field(alias="Body")
    conclusion_comments: List[str] = Field(alias="Conclusion")
    misc_comments: List[str] = Field(alias="Misc")
    spelling_errors: List[str] = Field(alias="Spelling errors")
    
    marks: MarksBreakdown = Field(alias="Total mark")

# Root Model
class EvaluationOutput(BaseModel):
    feedback: FeedbackDetail

ocr_parser = JsonOutputParser(pydantic_object=Ocr)
evaluation_structure_parser = JsonOutputParser(pydantic_object= EvaluationStructure)
evaluation_output_parser = JsonOutputParser(pydantic_object= EvaluationOutput)


## Prompts

In [26]:
# GS1
gs1_text = """
<System Role>
You are an expert UPSC Evaluator for General Studies Paper 1. Your task is to generate a comprehensive grading rubric for the provided question.

Focus: Time, Space, Culture, and Social Dynamics.

Analysis Protocol:
Keyword Analysis: Identify the directive (e.g., 'Elucidate', 'Trace'). If it asks to 'Trace', ensure the rubric demands a chronological flow.

Structure:
- Introduction: Must provide historical context (time period) or geographical definition.
- Body: Break down into dimensions: Cultural, Social, Geographic factors.
- Conclusion: Summarize significance or relevance to modern India.

CRITICAL - YOU MUST IDENTIFY THESE VALUE-ADDS:
- For History: Specific dates/timelines, names of historians/personalities, ancient terms (e.g., 'Muqtis', 'Iqta'), and archaeological sites.
- For Geography: Explicit mention of 'Map Locations' to draw, physical phenomena (e.g., 'Coriolis force'), and recent geophysical events (e.g., 'Joshimath sinking').
- For Society: Sociological concepts (e.g., 'Sanskritization', 'Regionalism'), names of sociologists (e.g., M.N. Srinivas), and census trends.

CONSTRAINT:
Do NOT look for Constitutional Articles or Economic Surveys here unless strictly necessary. Focus on Context, Culture, and Geography.
</System Role>

<User Input>
Question to Evaluate: {question}
Total_Marks : {total_marks}
</User Input>
"""
# GS2
gs2_text = """
<System Role>
You are an expert UPSC Evaluator for General Studies Paper 2. Your task is to generate a comprehensive grading rubric for the provided question.

Focus: The Constitution, Laws, and Supreme Court Judgments.

Analysis Protocol:
Keyword Analysis: If the question says 'Critically Examine', the rubric must demand a section on 'Challenges/Criticism' of the policy or law.

Structure:
- Introduction: Must start with a Constitutional Article, Amendment number, or a recent Supreme Court judgment.
- Body: Break down into Legislative, Executive, and Judicial aspects.
- Conclusion: Must be visionary, citing DPSP or Fundamental Duties.

CRITICAL - YOU MUST IDENTIFY THESE VALUE-ADDS:
- Articles: Precise Constitutional Articles (e.g., Art 21, Art 356) are mandatory.
- Judgments: Landmark Supreme Court cases (e.g., 'Kesavananda Bharati', 'Puttaswamy case').
- Committees: 2nd ARC Reports, Punchhi Commission, Law Commission Reports.
- For IR: Name specific treaties (e.g., 'Indus Water Treaty'), doctrines (e.g., 'Gujral Doctrine'), or groupings (e.g., 'G20', 'Global South').

Penalty:
Penalize answers that write generic points without citing Articles or Commissions.
</System Role>

<User Input>
Question to Evaluate: {question}
Total_Marks : {total_marks}
</User Input>
"""

# GS3
gs3_text = """
<System Role>
You are an expert UPSC Evaluator for General Studies Paper 3. Your task is to generate a comprehensive grading rubric for the provided question.

Focus: Data, Reports, Budget, and Technical Specifics.

Analysis Protocol:
Keyword Analysis: Focus on 'Analyze' or 'Suggest Measures'. The rubric must prioritize solutions and the 'Way Forward'.

Structure:
- Introduction: Must define the term or provide current statistics (e.g., 'Inflation at 6%').
- Body: Sectoral analysis (Agriculture, Industry, Services).
- Conclusion: Sustainable Development Goals (SDGs) or Prime Minister's vision (e.g., 'Amrit Kaal').

CRITICAL - YOU MUST IDENTIFY THESE VALUE-ADDS:
- Data & Reports: Economic Survey findings, Budget allocations, NITI Aayog reports, World Bank data.
- Schemes: Specific Government schemes (e.g., 'PM Gati Shakti', 'PLI Scheme').
- Environment: COP summits, Net Zero targets, Environmental Acts (WPA 1972).
- Science/Security: Technical keywords (e.g., 'Generative AI', 'Cyber-Physical Systems') and security agencies/forces.

Penalty:
Penalize answers that are purely theoretical. The rubric must demand Facts, Figures, and Scheme Names.
</System Role>

<User Input>
Question to Evaluate: {question}
Total_Marks : {total_marks}
</User Input>
"""



# Ocr Prompt
ocr_prompt = PromptTemplate(
    template= """
Role: You are a highly precise Optical Character Recognition (OCR) expert specializing in messy or cursive handwritten academic documents.
Task: Analyze the provided image of a student's essay and extract the content according to these strict rules:

1. The Question: Identify the specific question or prompt the student is answering. If it is not explicitly written, infer the most likely question based on the content of the essay.
2. The Transcription: Transcribe the body of the essay with 100% literal accuracy.
Do NOT fix spelling mistakes, punctuation, or grammatical errors.
Maintain original line breaks where possible to preserve the structure.

3. Diagrams: If you encounter a drawing, chart, or diagram, do not attempt to transcribe the text inside it. Instead, insert a placeholder: [Diagram: Provide a brief, one-sentence description of what the diagram depicts].

{format_instructions}
""",
input_variables=[],
partial_variables={"format_instructions": ocr_parser.get_format_instructions()}
)


# evaluation Prompt 
evaluation_prompt = PromptTemplate(
    template= """
Act as a strict UPSC Mains Examiner. Evaluate the answer based on the following specific criteria:

Phase 1: Compliance Check
Directive Word Analysis: Did the candidate follow the directive? (e.g., If the question asks to "Critically Analyze," did they provide both challenges and positives? If "Enumerate," did they simply list points?)
Word Count & Presentation: Is the answer within the limit? Are headings and subheadings used effectively?

Phase 2: Line-by-Line Auditing
Fact Verification: Cross-reference all statistics, Articles, and Case Laws. Deduct marks for factual inaccuracies.
Relevance: Flag any sentences that are "filler" or generic. Points must be specific to the question asked.

Phase 3: Component Scoring

Introduction (10%): Must be definition-based or data-based.

Body (70%): Look for diversity of points (PESTLE approach: Political, Economic, Social, Tech, Legal, Environmental).

Conclusion (20%): Must be forward-looking or solution-oriented.

Model Structure of the answer is given below. Do not compare it as it is with the Student Answer, The Model Structure of the answer is just for the reference
{model_structure} \n\n

Check the 'Student' answer below
{answer} \n \n

{format_instructions}
""",
input_variables=['model_structure' , 'answer'],
partial_variables={'format_instructions' : evaluation_output_parser.get_format_instructions()}
)

In [27]:
GS_prompt = {"GS1" : gs1_text,
             "GS2" : gs2_text,
             "GS3" : gs3_text}

In [28]:
print(ocr_prompt.format())


Role: You are a highly precise Optical Character Recognition (OCR) expert specializing in messy or cursive handwritten academic documents.
Task: Analyze the provided image of a student's essay and extract the content according to these strict rules:

1. The Question: Identify the specific question or prompt the student is answering. If it is not explicitly written, infer the most likely question based on the content of the essay.
2. The Transcription: Transcribe the body of the essay with 100% literal accuracy.
Do NOT fix spelling mistakes, punctuation, or grammatical errors.
Maintain original line breaks where possible to preserve the structure.

3. Diagrams: If you encounter a drawing, chart, or diagram, do not attempt to transcribe the text inside it. Instead, insert a placeholder: [Diagram: Provide a brief, one-sentence description of what the diagram depicts].

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, expl

In [29]:
import os
import mimetypes

flash_model = ChatGoogleGenerativeAI(model='gemini-2.5-flash', temperature= 1)
pro_model = ChatGoogleGenerativeAI(model = 'gemini-2.5-pro' , temperature= 0.5)

class AnswerEvaluation:

    def __init__(self, total_marks: int, subject: str , flash_model, pro_model):
        self.flash_model = flash_model
        self.pro_model = pro_model
        
        if total_marks not in [10, 15, 20]:
            raise ValueError("Marks must be 10, 15, or 20")
        self.total_marks = total_marks

        if subject not in ["GS1", "GS2", "GS3"]:
            raise ValueError("Subject must be GS1, GS2, or GS3")
        self.subject = subject 

    def prepare_content(self, files_path: List[str]):
        """Helper to create the image/pdf payload"""
        content_blocks = [
            {"type": "text", "text": ocr_prompt.format()}
        ]

        for path in files_path:
            if not os.path.exists(path):
                raise FileNotFoundError(f"File not found: {path}")
            
            
            mime_type, _ = mimetypes.guess_type(path)
            if mime_type is None:
                if path.lower().endswith(('.jpg', '.jpeg')): mime_type = 'image/jpeg' 
                elif path.lower().endswith('.png'): mime_type = 'image/png'
                else: mime_type = 'application/octet-stream' 

            # 2. Encode
            with open(path, 'rb') as f:
                file_bytes = f.read()
                base64_data = base64.b64encode(file_bytes).decode("utf-8")

            # 3. Append to list
            content_blocks.append({
                "type": "media",
                "file_uri": None,
                "data": base64_data,
                "mime_type": mime_type
            })

        return content_blocks
        
    def ocr(self, files_path: List[str]):
        """Extracts text and question from images"""
        print("Running OCR...")
        message_content = self.prepare_content(files_path)

        if len(message_content) > 1:
            msg = HumanMessage(content=message_content)

            try:
                response = self.flash_model.invoke([msg])
                parsed_output = ocr_parser.parse(response.content)
                return parsed_output
            except Exception as e:
                print(f"OCR Error: {e}")
                return None
        else:
            print("No valid files to process.")
            return None

    def keypoint_generator(self, question: str):
        """Generates the marking scheme"""
        print("Generating Rubric...")
        prompt = PromptTemplate(
            template=GS_prompt[self.subject] + "\n\n {format_instructions}",
            input_variables=['question', 'total_marks'],
            partial_variables={'format_instructions': evaluation_structure_parser.get_format_instructions()}
        )


        chain = prompt | self.flash_model | evaluation_structure_parser
        res = chain.invoke({'question': question, 'total_marks': self.total_marks})
        return res

    def feedback(self, structure: Dict, answer: str):
        """Generates the final evaluation"""
        print("Evaluating Answer...")
        chain = evaluation_prompt | self.flash_model | evaluation_output_parser
        
        # # Convert structure to string for the prompt context
        # structure_str = json.dumps(structure)
        
        res = chain.invoke({'model_structure': structure, 'answer': answer})
        return res

    def full_evaluation(self, file_paths: List[str]):
        """Orchestrates the entire pipeline"""
        
        # Step 1: OCR
        ocr_result = self.ocr(file_paths)
        if not ocr_result: return "OCR Failed"
        
        student_question = ocr_result['question']
        student_answer = ocr_result['answer']
        
        # Step 2: Generate Rubric
        rubric = self.keypoint_generator(student_question)
        
        # Step 3: Evaluate
        final_feedback = self.feedback(rubric, student_answer)
        
        # return final_feedback
        return {'ocr_res' : ocr_result,
                'rubric_res' : rubric,
                'feedback_res' : final_feedback}

In [30]:
import json
import pprint

# 1. Initialize for a specific subject and marks
evaluator = AnswerEvaluation(total_marks=10, subject="GS1" , flash_model= flash_model, pro_model=pro_model)

files = [
    r"e:\Upse_project\Images\answer.pdf"
    # r"e:\Upse_project\Images\new2.jpg"
]

result = evaluator.full_evaluation(files)

Running OCR...
Generating Rubric...
Evaluating Answer...


In [31]:
pprint.pprint(result['ocr_res'] , indent = 4)


{   'answer': "• India's potential growth depands on the interpelay\n"
              'of several factors such as Domestic demand, Saving,\n'
              'Investment, Technology, etc.\n'
              '\n'
              '# Why Domestic demand is most effective\n'
              '\n'
              '① If Demand is high in a Countery, which in\n'
              'turns mean more consumption. This will led to fewer\n'
              'Saving with bank. Interest & rate of loans with\n'
              'get high which will discourage further investment\n'
              'by Company: At last commpand Economy will get\n'
              'into stagfilation\n'
              '\n'
              '② If Demand is low→\n'
              '[Diagram: A cyclical diagram showing the effects of low demand, '
              'leading to more saving, less investment, falling GDP, and '
              'growing pessimism.]\n'
              '\n'
              '• Paradox of thrift. Even though Countries saving rate is\n'
    

In [32]:
pprint.pprint(result['rubric_res'] , indent = 4)

{   'grading_rubric': {   'body': {   'conclusion': {   'criteria': 'Conclude '
                                                                    'by '
                                                                    'synthesizing '
                                                                    'the '
                                                                    'multifaceted '
                                                                    'influences '
                                                                    'on '
                                                                    "India's "
                                                                    'growth '
                                                                    'trajectory. '
                                                                    'Reiterate '
                                                                    'the '
                                                   

In [16]:

pprint.pprint(result['feedback_res'] , indent = 4)

{   'feedback': {   'Body': [   "The argument regarding 'High Demand' is "
                                'flawed. High demand typically leads to '
                                'inflation, not directly to stagflation (which '
                                'combines inflation with stagnant growth and '
                                'high unemployment). The causal chain '
                                'presented is economically inaccurate.',
                                "The 'Paradox of Thrift' is correctly "
                                "identified and used in the context of 'Low "
                                "Demand'.",
                                "The concept of 'Moderate Demand' leading to a "
                                "'Virtuous Economic Cycle' is "
                                'well-articulated.',
                                "The mention of 'personal Investment led Model "
                                "(Economic Survey 2018-19)' is incorrect.

In [33]:
print(json.dumps(result['rubric_res'] , indent = 4))

{
    "question_text": "Among Several facton's fon India's potential growth, domestic demand is the most effective one. Do you agree? What are other factors that influence the growth trapetory of the country.",
    "max_marks": 10,
    "grading_rubric": {
        "introduction": {
            "marks": 1,
            "criteria": "Introduce the multi-faceted nature of India's growth trajectory, acknowledging the role of domestic demand while setting the stage for a holistic analysis encompassing historical, geographical, social, and cultural dimensions relevant to GS Paper 1."
        },
        "body": {
            "marks": 8,
            "dimensions": [
                {
                    "title": "Analysis of Domestic Demand (Historical, Cultural, Social Dimensions)",
                    "marks": 3,
                    "expected_elements": [
                        "A clear stance on whether domestic demand is the 'most effective' factor for India's growth.",
                      